<a href="https://colab.research.google.com/github/vrra/FGAN-Build-a-thon/blob/main/Notebooks2023/PDF_parsing_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install pypdf
!pip install langchain
!pip install pandas


In [20]:
from langchain.document_loaders import PyPDFLoader
import pandas as pd


In [21]:
loader = PyPDFLoader("/content/sample_data/T-REC-Y.Sup71-usecases.pdf")
pages = loader.load_and_split()

In [ ]:
simple_data_container = []
for i in range(len(pages)):
  print("Physical Page =",
      i,
      "logical page given by the loader = ",
      pages[i].metadata['page'],
      "of ",
      pages[i].metadata['source'],
      "content = ",
      pages[i].page_content)
  simple_data_container.append(
      [i,
       pages[i].metadata['source'],
       pages[i].page_content])
  print("*****************************")



In [31]:
#check whether a parquet file is already there
#if yes, create a df from it
#and append the data_container into it.

df = pd.DataFrame(simple_data_container,
                  columns=['page_number', 'document_source', 'page_content'])

In [ ]:
df.head(10)

In [34]:
df.to_parquet('/content/sample_data/p1_usecases.parquet')